In [1]:
import torch
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

In [2]:
# Load Train, Validation, Test Dataset
train_dataset = pd.read_csv('/kaggle/input/mp-data/MP_IN_adm_train.csv')
val_dataset = pd.read_csv('/kaggle/input/mp-data/MP_IN_adm_val.csv')
test_dataset = pd.read_csv('/kaggle/input/mp-data/MP_IN_adm_test.csv')

In [3]:
from torch.utils.data import DataLoader
from torch import nn

class EnsembleModel(nn.Module):
    def __init__(self, model1):
        super(EnsembleModel, self).__init__()
        self.model1 = model1

    def forward(self, input_ids, attention_mask):
        output1 = self.model1(input_ids, attention_mask=attention_mask)[0]
        avg_output = output1
        return avg_output

In [4]:

from transformers import DistilBertForSequenceClassification, DistilBertConfig, AdamW, get_linear_schedule_with_warmup

# create a student model
student_config = DistilBertConfig.from_pretrained('distilbert-base-uncased', 
                                                  num_labels=2, 
                                                  hidden_dropout_prob=0.2, 
                                                  attention_probs_dropout_prob=0.2)

student_model = DistilBertForSequenceClassification(student_config)

# set the temperature
temperature = 2.0

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [5]:
from transformers import AutoTokenizer

# Choose a tokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
# Apply the tokenizer to the training, validation, and test datasets
train_encodings = tokenizer(train_dataset['text'].tolist(), truncation=True, padding=True, max_length = 512)
val_encodings = tokenizer(val_dataset['text'].tolist(), truncation=True, padding=True,  max_length = 512)
test_encodings = tokenizer(test_dataset['text'].tolist(), truncation=True, padding=True , max_length = 512)

In [7]:
# Create a Dataset for PyTorch
class LosDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [8]:
train_dataset = LosDataset(train_encodings, train_dataset['hospital_expire_flag'].tolist())
val_dataset = LosDataset(val_encodings, val_dataset['hospital_expire_flag'].tolist())
test_dataset = LosDataset(test_encodings, test_dataset['hospital_expire_flag'].tolist())

In [9]:
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score
from tqdm import tqdm
from torch import nn
import numpy as np

# Create the ensemble model
ensemble_model = EnsembleModel(student_model)

In [10]:
import os

# list all files in the current directory
files = os.listdir('.')

# filter the ones that start with 'CORE_baseline'
core_models = [f for f in files if f.startswith('distilBERT_baseline_MP')]

if core_models:
    print("Found models starting with 'distilBERT_baseline_MP':")
    for model in core_models:
        print(model)
        
    # get the first (and supposedly only) model
    model_path = core_models[0]

    # load the model state
    ensemble_model.load_state_dict(torch.load(model_path))
    print("Loaded Model")
else:
    print("No models found starting with 'distilBERT_baseline_MP'.")

No models found starting with 'distilBERT_baseline_MP'.


In [11]:
# Push the model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ensemble_model = ensemble_model.to(device)

In [12]:
ensemble_model

EnsembleModel(
  (model1): DistilBertForSequenceClassification(
    (distilbert): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_features=768, bias=True)
            )
            (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (

In [13]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [14]:
epochs = 200
best_roc_auc = 0.0
min_delta = 0.0001
early_stopping_count = 0
early_stopping_patience = 3
gradient_accumulation_steps = 10
best_model_path = "best_model.pth"

# Set the optimizer
optimizer = AdamW(ensemble_model.parameters(), lr=1e-5, weight_decay=0.01)

# Set the scheduler
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=50, 
    num_training_steps=len(train_loader) * epochs // gradient_accumulation_steps
)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [15]:
from torch.nn import functional as F
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Training
for epoch in range(epochs):
    ensemble_model.train()
    train_loss = 0
    for step, batch in enumerate(tqdm(train_loader)):
        optimizer.zero_grad() if step % gradient_accumulation_steps == 0 else None
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = ensemble_model(input_ids, attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        (loss / gradient_accumulation_steps).backward()
        train_loss += loss.item()
        if (step + 1) % gradient_accumulation_steps == 0 or (step + 1) == len(train_loader):
            optimizer.step()
            scheduler.step()

    ensemble_model.eval()
    val_loss = 0
    val_preds = []
    val_labels = []
    with torch.no_grad():
        for batch in tqdm(val_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = ensemble_model(input_ids, attention_mask)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            val_loss += loss.item()
            val_preds.append(F.softmax(outputs, dim=1).cpu().numpy())
            val_labels.append(labels.cpu().numpy())
            
    val_preds = np.concatenate(val_preds)
    val_labels = np.concatenate(val_labels)
    val_loss /= len(val_loader)
    train_loss /= len(train_loader)
    print(f'Epoch: {epoch+1}/{epochs}, Training Loss: {train_loss}, Validation Loss: {val_loss}')

    # Calculate metrics
    val_preds_class = np.argmax(val_preds, axis=1)
    accuracy = accuracy_score(val_labels, val_preds_class)
    recall = recall_score(val_labels, val_preds_class)
    precision = precision_score(val_labels, val_preds_class)
    f1 = f1_score(val_labels, val_preds_class)
    roc_auc = roc_auc_score(val_labels, val_preds[:, 1])

    print(f'Accuracy: {accuracy}, Recall: {recall}, Precision: {precision}, F1: {f1}, Roc Auc: {roc_auc}')

    # Implement early stopping
    if epoch > 0 and roc_auc - best_roc_auc < min_delta:
        early_stopping_count += 1
        print(f'EarlyStopping counter: {early_stopping_count} out of {early_stopping_patience}')
        if early_stopping_count >= early_stopping_patience:
            print('Early stopping')
            break
    else:
        best_roc_auc = roc_auc
        early_stopping_count = 0
        torch.save(ensemble_model.state_dict(), f"distilBERT_baseline_MP_epoch_{epoch}roc_{best_roc_auc}.pth")

100%|██████████| 154/154 [01:23<00:00,  1.85it/s]
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 1/200, Training Loss: 0.42624114210000036, Validation Loss: 0.33751997061364064
Accuracy: 0.8944580277098615, Recall: 0.0, Precision: 0.0, F1: 0.0, Roc Auc: 0.5982352837705913


100%|██████████| 154/154 [01:23<00:00,  1.85it/s]
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 2/200, Training Loss: 0.334605549183269, Validation Loss: 0.3351525234711635
Accuracy: 0.8944580277098615, Recall: 0.0, Precision: 0.0, F1: 0.0, Roc Auc: 0.6079348466592203


100%|██████████| 154/154 [01:23<00:00,  1.85it/s]
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Epoch: 12/200, Training Loss: 0.2294359154781772, Validation Loss: 0.28470618031048156
Accuracy: 0.8983292583537082, Recall: 0.12548262548262548, Precision: 0.5855855855855856, F1: 0.20667726550079493, Roc Auc: 0.8151625755270401
EarlyStopping counter: 1 out of 3


100%|██████████| 154/154 [01:23<00:00,  1.85it/s]


Epoch: 13/200, Training Loss: 0.22029451837828085, Validation Loss: 0.2952255007822986
Accuracy: 0.8936430317848411, Recall: 0.16988416988416988, Precision: 0.4888888888888889, F1: 0.25214899713467054, Roc Auc: 0.8148402388721296
EarlyStopping counter: 2 out of 3


100%|██████████| 154/154 [01:23<00:00,  1.85it/s]

Epoch: 14/200, Training Loss: 0.22063848987860138, Validation Loss: 0.2847086194322094
Accuracy: 0.8909942950285249, Recall: 0.2084942084942085, Precision: 0.463519313304721, F1: 0.28761651131824234, Roc Auc: 0.8150420840625853
EarlyStopping counter: 3 out of 3
Early stopping


In [16]:
import os

# list all files in the current directory
files = os.listdir('.')

# filter the ones that start with 'CORE_baseline'
core_models = [f for f in files if f.startswith('distilBERT_baseline_MP')]

if core_models:
    print("Found models starting with 'distilBERT_baseline_MP':")
    for model in core_models:
        print(model)
        
    # get the first (and supposedly only) model
    model_path = core_models[0]

    # load the model state
    ensemble_model.load_state_dict(torch.load(model_path))
    print("Loaded Model")
else:
    print("No models found starting with 'distilBERT_baseline_MP'.")

/tmp/ipykernel_30/2612778148.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ensemble_model.load_state_dict(torch.load(model_path))


Found models starting with 'distilBERT_baseline_MP':
distilBERT_baseline_MP_epoch_3roc_0.6680446961768145.pth
distilBERT_baseline_MP_epoch_9roc_0.8151137632914398.pth
distilBERT_baseline_MP_epoch_4roc_0.7607756308211889.pth
distilBERT_baseline_MP_epoch_10roc_0.8162729439494816.pth
distilBERT_baseline_MP_epoch_5roc_0.7903716765903553.pth
distilBERT_baseline_MP_epoch_1roc_0.6079348466592203.pth
distilBERT_baseline_MP_epoch_2roc_0.6218821734197588.pth
distilBERT_baseline_MP_epoch_8roc_0.8129128151907196.pth
distilBERT_baseline_MP_epoch_6roc_0.8041138600364113.pth
distilBERT_baseline_MP_epoch_0roc_0.5982352837705913.pth
distilBERT_baseline_MP_epoch_7roc_0.8100698322793995.pth
Loaded Model


In [17]:
from torch.nn import functional as F
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Put the model in evaluation mode
ensemble_model.eval()

# Initialize lists to store predictions and true labels
test_preds = []
test_labels = []

# Iterate over test data
with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = ensemble_model(input_ids, attention_mask)
        test_preds.append(F.softmax(outputs, dim=1).cpu().numpy())
        test_labels.append(labels.cpu().numpy())

100%|██████████| 307/307 [02:46<00:00,  1.84it/s]


In [18]:
test_preds = np.concatenate(test_preds)
test_labels = np.concatenate(test_labels)

# Calculate metrics
test_preds_class = np.argmax(test_preds, axis=1)
accuracy = accuracy_score(test_labels, test_preds_class)
recall = recall_score(test_labels, test_preds_class)
precision = precision_score(test_labels, test_preds_class)
f1 = f1_score(test_labels, test_preds_class)
roc_auc = roc_auc_score(test_labels, test_preds[:, 1])

print(f'Accuracy: {accuracy}, Recall: {recall}, Precision: {precision}, F1: {f1}, Roc Auc: {roc_auc}')

Accuracy: 0.8956424353492161, Recall: 0.0, Precision: 0.0, F1: 0.0, Roc Auc: 0.6943097009235409


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
